<a href="https://colab.research.google.com/github/royam0820/crewai/blob/main/L7_job_application_crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L7: Build a Crew to Tailor Job Applications

The notebook titled "L7: Build a Crew to Tailor Job Applications" consists in the process of creating a team of AI agents to assist in tailoring job applications.

In this lesson, you will built your first multi-agent system.

## Accesssing the Google Drive

We will use it to upload the fake resume `fake_resume.md` that is used in this notebook. The fake resume serves as a placeholder or template for testing and demonstrating the functionality of the AI agents and the overall workflow. It acts as a template used to ensure that the outputs generated by the agents (such as the tailored resume and interview materials) are what is expected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Library Installation

In [ ]:
!pip install crewai==0.28.8
!pip install crewai_tools==0.1.6
!pip install langchain_community==0.0.29 # this is formatted as code
!pip install utils

## Warnings Suppression

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Importing Required Classes

In [ ]:
from crewai import Agent, Task, Crew
import os
from google.colab import userdata

## Setting API Keys and LLM

In [ ]:
openai_api_key = userdata.get("OPENAI_API_KEY")
serper_api_key = userdata.get("SERPER_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = serper_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

## Defining Tools

In [ ]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  3.01it/s]


NB: `MDXSearchTool`: The MDX Search Tool is a component of the crewai_tools package aimed at facilitating advanced market data extraction.
The `SerperDevTool` is designed to perform a semantic search for a specified query from a text's content across the internet. It utilizes the serper.dev API to fetch and display the most relevant search results based on the query provided by the user.

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))


# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. He is highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Noah's significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Noah has transformed the engineering division into a key revenue pillar for DataKernel, rapidly expanding the customer base from inception to a robust community.
- He spearheaded the integration of cutting-edge AI technologies and the use of scalable vector databases, which significantly enhanced the product's capabilities and market positioning.
- Under his leadership, the team has seen a substantial growth in skill development, with a focus on achieving strategic project goals that have significantly influenced the company's direction.
- Noah also played a critical role in defining the company’s long-term strategic initiatives, particularly in adopting AI technologies that have set new benchmarks within the industry.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed the engineering strategy and operations in close collaboration with C-level executives, playing a pivotal role in shaping the company's technological trajectory.
- Managed diverse teams across multiple time zones in North America and Europe, creating an environment of transparency and mutual respect which enhanced team performance and morale.
- His initiatives in recruiting, mentoring, and retaining top talent have been crucial in fostering a culture of continuous improvement and high performance.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Noah founded InnovPet, a startup focused on innovative IoT solutions for pet care, including a revolutionary GPS tracking collar that significantly enhanced pet safety and owner peace of mind.
- He was responsible for overseeing product development from concept through execution, working closely with engineering teams and marketing partners to ensure a successful market entry.
- Successfully set up an advisory board, established production facilities overseas, and navigated the company through a successful initial funding phase, showcasing his leadership and entrepreneurial acumen.
- Built the initial version of the product leveraging MongoDB

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Noah was instrumental in formulating and executing strategic plans that enhanced inter-departmental coordination and trust, leading to better project outcomes.
- He managed multiple engineering teams, fostering a culture that balances productivity with innovation, and implemented goal-setting frameworks that aligned with the company's long-term goals.
- Was a bery hands on manager using ruby on rails and react to build out a new product.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led the formation and development of a high-performance engineering team that was pivotal in increasing company revenue by 500% within two years.
- His leadership in integrating data analytics into business decision-making processes led to the development of a predictive modeling tool that revolutionized customer behavior analysis.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API that significantly improved the functionality used by a large engineering team and thousands of users, enhancing overall system performance and user satisfaction.
- Implemented several critical enhancements, including advanced caching strategies that drastically reduced response times and system loads.

### BetCraft: CTO — 2013 - 2015
- Led the technology strategy post-Series A funding, directly reporting to the board and guiding the company through a phase of significant technological advancement and network expansion.
- His strategic initiatives and partnerships significantly improved platform performance and expanded the company's market reach.
- Helped build his initial product using both React and Angular and got pretty good at it.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is an ideal candidate for senior executive roles, particularly in companies seeking leadership with a robust blend of technical and strategic expertise.


## Defining Agents

### Researcher

Extracts job requirements.

In [ ]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

### Profile

Compiles a comprehensive profile.

In [ ]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

### Resume Strategist

 Aligns the resume with job requirements.

In [ ]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

### Interview Preparer

Develops interview materials.

In [ ]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Defining Tasks

### Researcher Task

Extracts job requirements from the job posting.

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

### Profiler Task

Compiles a comprehensive profile from provided data.

In [ ]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

### Resume Strategist Task

Aligns the resume with job requirements.

In [ ]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

### Interview Preparer Task

 Develops interview materials.

In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Assembling the Crew and Executing the Workflow

In [ ]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Inputs and Execution

- Set the inputs for the execution of the crew.

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

### The Tailored Resume

- Dislplay the generated `tailored_resume.md` file.

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a visionary Software Engineering Leader with 18 years of experience in the tech industry, proficient in full-stack development including front-end and back-end technologies. He is skilled in HTML, CSS, JavaScript, Ruby, Python, and Elixir, and has extensive experience with frameworks like React, Angular, and Node.js. His expertise in managing both remote and in-office teams, coupled with his strong background in AI and data science, allows him to develop and deploy scalable software solutions effectively.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Transformed the engineering division to focus on scalable software solutions, incorporating front-end and back-end developments that enhanced the product's capabilities and market positioning.
- Integrated cutting-edge AI technologies and scalable vector databases, aligning with modern full-stack development practices.
- Fostered a team environment that emphasized skill development in multiple coding languages and frameworks, crucial for maintaining efficient and responsive software applications.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed engineering strategies that included the full software development life cycle, ensuring the creation of robust and secure software solutions.
- Managed a diverse team, focusing on ideating software solutions that aligned with business objectives and market needs.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Oversaw the development of an IoT solution from concept to execution, ensuring the application's front-end was user-friendly while maintaining robust back-end database management using MongoDB.

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Hands-on management of projects using Ruby on Rails and React, focusing on building features and applications that were both functional and aesthetically appealing.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led development projects that significantly increased company revenue through strategic enhancements in software applications and data integration.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized APIs and implemented advanced caching strategies, improving system responsiveness and efficiency, essential for high-load software environments.

### BetCraft: CTO — 2013 - 2015
- Spearheaded the development of the company’s flagship products using React and Angular, demonstrating strong technical expertise in front-end development and user interface design.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

This resume has been tailored to highlight Noah's proficiency in both the technical and leadership aspects required for the Full Stack Engineer position at AI Fund, focusing on his hands-on experience with front-end and back-end technologies, his leadership in developing scalable solutions, and his strategic approach to software development.

### The Interview Questions

- Dislplay the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))

**Interview Preparation Document for Noah Williams - Full Stack Engineer Position at AI Fund**

**Technical Skills and Experience:**

1. **Question:** "Can you discuss a project where you utilized both front-end and back-end technologies? How did you ensure these elements worked seamlessly together?"
   **Talking Point:** Highlight your work with crewAI and machinery, showcasing your ability to integrate multiple technologies (Python, Elixir, Ruby) across full-stack development.

2. **Question:** "You have experience with a variety of coding languages and frameworks. How do you decide which technologies are best suited for a particular project?"
   **Talking Point:** Reference your strategic decisions at DataKernel and InnovPet, emphasizing how you align technology choices with business objectives and project requirements.

3. **Question:** "What challenges have you faced while working with AI technologies and big data, and how have you overcome them?"
   **Talking Point:** Discuss the integration of AI technologies and scalable vector databases at DataKernel, demonstrating your problem-solving skills and technical proficiency.

**Project Management and Leadership:**

4. **Question:** "As a leader, how do you keep your team motivated and ensure projects stay on track?"
   **Talking Point:** Talk about your leadership style at EliteDevs and DataKernel, focusing on team skill development and maintaining high efficiency.

5. **Question:** "Can you give an example of a complex project you managed from conception to deployment? What were the key success factors?"
   **Talking Point:** Describe your role in orchestrating the IoT solution at InnovPet, detailing your approach to project management and cross-functional collaboration.

**Strategic Approach and Problem Solving:**

6. **Question:** "Describe a time when you had to innovate with limited resources. What was the outcome?"
   **Talking Point:** Use your experience founding InnovPet and developing its IoT solution to illustrate your ability to be scrappy and innovative under constraints.

7. **Question:** "How do you approach data privacy and security in your projects, especially when handling consumer applications?"
   **Talking Point:** Discuss the keeper project and its focus on authentication solutions, showcasing your commitment to secure software practices.

**Cultural Fit and Personal Insights:**

8. **Question:** "How do you align your technical decisions with a company's strategic goals, especially in a fast-paced, early-stage environment like AI Fund?"
   **Talking Point:** Reflect on your experiences at crewAI and DataKernel, where aligning technical strategy with business goals was crucial for growth.

9. **Question:** "AI Fund values accountability and ownership. Can you provide an example where you demonstrated these qualities?"
   **Talking Point:** Talk about your role at BetCraft as CTO and how you took ownership of the flagship product development, leading to significant business impacts.

**Additional Tips for the Interview:**
- Be prepared to discuss specific technical details of your projects, especially those that relate directly to the technologies and methodologies favored at AI Fund.
- Emphasize your leadership and communication skills, as these are highly valued in the role you are applying for.
- Reflect on how your strategic vision in past roles can translate into success at AI Fund, particularly in developing scalable and innovative software solutions.

This preparation document is designed to highlight your strengths and experiences that directly align with the job requirements at AI Fund, helping you to make a strong impression during your interview.

# Conclusion

This notebook demonstrated the power and flexibility of using AI to streamline and enhance the job application process. By automating key aspects of tailoring applications, we can save time and ensure a higher degree of personalization and relevance, ultimately increasing the chances of success in the job market.
